## DWaveCliqueSampler, EmbeddingComposite

In [1]:
#!pip3 install dwave-system

In [2]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
import itertools
import random
import matplotlib.pyplot as plt
from dwave.system.samplers import DWaveCliqueSampler
from dwave.system.composites import EmbeddingComposite
from dwave.embedding.chain_strength import uniform_torque_compensation
import dimod
import timeit

In [3]:
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']), #0
        np.dot(df['Y'], df['SEX']),      #1
        np.dot(df['Y'], df['AOP']),    #2  
    ])     

In [4]:
def make_Hamiltonian(df):
    t_list = calc_marginals(df)
    N=len(df)
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    num_Y = t_list[0]**2 #数字の二乗
    
    SEX = df['SEX'].iloc
    lin_SEX  = [(SEX[i] - 2 * t_list[1]) * SEX[i] for (i, _) in dup_list]
    quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]
    num_SEX  = t_list[1]**2
    
    AOP = df['AOP'].iloc
    lin_AOP = [(AOP[i] - 2 * t_list[2]) * AOP[i] for (i, _) in dup_list]
    quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]
    num_AOP = t_list[2]**2
    
    #lin
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    
    #quad
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    
    #num
    num = num_Y + num_SEX + num_AOP
    
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [5]:
def find_valid_y_1(df, num_reads):                                                        
    
    # dimod sampler
    dw_sampler = DWaveCliqueSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    
    #bqm
    bqm = make_Hamiltonian(df)

    #method
    res = qa_sampler.sample(bqm, chain_break_fraction=True, num_reads=num_reads)
    print(res)
    
    valid_y_list = []                                                           
    valid_y_num = 0                                                             
    for y_info in list(res.record):
        if y_info[1] == 0.:
            valid_y_num += 1                                        
            valid_y_list.append(list(y_info[0])) 
                                                      
    return valid_y_list, valid_y_num       

In [6]:
def find_valid_y_2(df, num_reads):                                                        
    DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    
    # dimod sampler
    dw_sampler = DWaveCliqueSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = DWAVE_TOKEN
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
                                                                                                                                                                                                                                               
    #bqm
    bqm = make_Hamiltonian(df)

    #method
    res = qa_sampler.sample(bqm, chain_break_fraction=False, num_reads=num_reads)
    print(res)
    
    valid_y_list = []                                                           
    valid_y_num = 0                                                             
    for y_info in list(res.record):
        if y_info[1] == 0.0:
            valid_y_num += 1                                        
            valid_y_list.append(list(y_info[0]))
                                                      
    return valid_y_list, valid_y_num

In [7]:
def find_valid_y_3(df, num_reads, max_chain_strength, chain_break_fraction):                                                        
    DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    
    # dimod sampler
    dw_sampler = DWaveCliqueSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = DWAVE_TOKEN
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = []                                                           
    valid_y_num = 0     
    
    bqm = make_Hamiltonian(df)

    for chain_strength in range(0, max_chain_strength):
        res = qa_sampler.sample(
            bqm, chain_strength = chain_strength, 
            chain_break_fraction=chain_break_fraction, num_reads=num_reads)
        print(res)
                                                                   
        for y_info in list(res.record):
            if y_info[1] == 0.:
                valid_y = list(y_info[0])
                if len(valid_y_list)==0:
                    valid_y_num += 1                                        
                    valid_y_list.append(valid_y)                    
                    print('perfect')

                elif all([valid_y != p for p in valid_y_list]):
                    valid_y_num += 1                                    
                    valid_y_list.append(valid_y)                  
                    print('perfect')
                    
                valid_y_num += 1                                        
                valid_y_list.append(valid_y)
                                                      
    return valid_y_list, valid_y_num

In [8]:
def find_valid_y_4(df, num_reads, max_chain_strength, chain_break_fraction):                                                        
    DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    
    # dimod sampler
    dw_sampler = DWaveCliqueSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = DWAVE_TOKEN
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    bqm = make_Hamiltonian(df)

    for chain_strength in range(0, max_chain_strength):
        res = qa_sampler.sample(
            bqm, chain_strength = chain_strength, 
            chain_break_fraction=chain_break_fraction, num_reads=num_reads)
        print(res)
        
        valid_y_list[chain_strength] = []                                                           
        valid_y_num[chain_strength] = 0                                                             
        for y_info in list(res.record):
            if y_info[1] == 0.0:
                valid_y_num[chain_strength] += 1                                        
                valid_y_list[chain_strength].append(list(y_info[0]))
                                                      
    return valid_y_list, valid_y_num

### [qa_sampler.sampleのパラメータ](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/generated/dwave.system.composites.AutoEmbeddingComposite.sample.html)
- chain_strength(float)
- chain_break_fraction(bool):Add a chain_break_fraction field to the unembedded dimod.SampleSet with the fraction of chains broken before unembedding.
- num_reads
- return_embedding(bool):詳細を表示するか否か

### 16bitで確認

In [9]:
df = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)

In [10]:
calc_marginals(df)#t1 = 6

array([8, 4, 4])

In [11]:
num_reads = 100
find_valid_y_1(df, num_reads)

ValueError: no structured sampler found

In [12]:
num_reads = 100
find_valid_y_2(df, num_reads)

ValueError: no structured sampler found

### ここまでのまとめ
- DwaveCliqueSampler & Embeddingはエラーが起こり実行できない

In [13]:
find_valid_y_3(df, num_reads=100, max_chain_strength=10, chain_break_fraction=True)

ValueError: no structured sampler found

In [32]:
num_reads = 100
find_valid_y_4(df, num_reads)#chain_strength == 4, 7のとき良い。chain_break_fraction=falseのほうがいい？それとも誤差？

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
4   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       4   0.125
5   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       2   0.125
6   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0       9   0.125
7   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      31   0.125
9   0  0  0  0  0  1  0  0  0  0  1  1  1  1  1  1    1.0       1  0.1875
10  0  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1    1.0       2  0.1875
17  1  1  1  1  0  0  1  1  0  0  0  0  0  1  0  0    1.0       1   0.125
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       3  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      12  0.1875
2   0  0  0  0  0  1  0  0  1  1  1  0  0  1  1  1    2.0       1  0.1875
3   0  0  0  0  0  1  0  0  0  1  1  1  0  1  1  1    2.0       1  0.1875
11  0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  0    2.0       1   0.125
12  0  0  0  0  0  1  0  0  1  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
4   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       2   0.125
5   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       1   0.125
6   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      28   0.125
7   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      13   0.125
8   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0       1  0.1875
10  0  0  0  0  0  1  0  0  0  0  1  1  1  1  1  1    1.0       2  0.1875
11  0  0  0  0  1  0  0  0  1  0  1  0  1  1  1  1    1.0       1  0.1875
12  0  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1    1.0       2  0.1875
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       1  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      27  0.1875
2   0  0  0  0  0  1  0  0  1  1  1  0  0  1  1  1    2.0       2  0.1875
3   0  0  0  0  0  1  0  0  0  1  1  1  0  1  1  1    2.0       1  0.1875
17  1  1  0  1  0  1  1  1  0  0  1  0

({0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []},
 {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0})

In [36]:
find_valid_y_3(df, num_reads=100, max_chain_strength=10, chain_break_fraction=False)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
3   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       1   0.125
4   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       6   0.125
5   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      24   0.125
6   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      28   0.125
7   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0       1  0.1875
8   0  0  0  0  0  1  0  0  0  0  1  1  1  1  1  1    1.0       1  0.1875
9   0  0  0  0  1  0  0  0  0  0  1  1  1  1  1  1    1.0       1  0.1875
10  0  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1    1.0       2  0.1875
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       3  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      21  0.1875
2   0  0  0  0  0  1  0  0  1  1  1  0  0  1  1  1    2.0       1  0.1875
11  0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0       1    0.25
12  0  0  0  0  0  1  0  0  1  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
3   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       9   0.125
4   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       3   0.125
5   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      14   0.125
6   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      28   0.125
7   0  0  0  0  1  0  0  0  1  1  1  1  0  1  1  1    1.0       1   0.125
8   0  0  0  0  0  1  0  0  1  1  1  1  0  1  1  1    1.0       2   0.125
10  0  0  0  0  1  0  0  0  0  0  1  1  1  1  1  1    1.0       1  0.1875
11  0  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1    1.0       2  0.1875
12  0  0  0  0  0  1  0  0  1  1  0  0  1  1  1  1    1.0       1  0.1875
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       5  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      12  0.1875
2   0  0  0  0  0  1  0  0  0  1  1  1  0  1  1  1    2.0       1  0.1875
9   0  0  0  0  0  1  0  0  0  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
2   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       6   0.125
3   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       3   0.125
4   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      16   0.125
5   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      23   0.125
7   0  0  1  0  0  1  0  0  0  1  1  0  1  1  1  1    1.0       1    0.25
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       8  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      31  0.1875
6   0  0  0  0  0  0  0  0  1  1  1  0  1  1  1  1    2.0       1  0.1875
8   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  0    2.0       1   0.125
9   1  1  1  1  0  0  1  1  0  0  0  1  0  0  0  0    3.0       5  0.1875
10  1  1  1  1  0  0  1  1  1  0  0  0  0  0  0  0    3.0       4  0.1875
11  1  1  1  1  0  0  1  1  0  0  0  0  1  0  0  0    3.0       1  0.1875
['BINARY', 12 rows, 100 samples, 16 va

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
4   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0      11   0.125
5   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       6   0.125
6   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      11   0.125
7   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      21   0.125
8   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       1  0.1875
9   0  0  0  0  1  0  0  0  1  0  1  0  1  1  1  1    1.0       1  0.1875
10  0  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1    1.0       1  0.1875
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       6  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      28  0.1875
2   0  0  0  0  0  1  0  0  1  1  1  0  0  1  1  1    2.0       2  0.1875
3   0  0  0  0  0  1  0  0  0  1  1  1  0  1  1  1    2.0       1  0.1875
11  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1  1    2.0       1   0.125
12  1  1  1  0  1  0  1  1  0  0  0  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
2   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       4   0.125
3   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       7   0.125
4   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      13   0.125
5   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      15   0.125
6   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0       1  0.1875
7   0  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1    1.0       1  0.1875
8   0  0  0  0  0  1  0  0  1  1  0  0  1  1  1  1    1.0       1  0.1875
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0      14  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      30  0.1875
9   0  0  0  0  0  1  0  0  0  1  1  1  1  0  1  1    2.0       1   0.125
13  1  1  1  1  0  0  1  1  0  1  0  0  0  0  0  0    2.0       1  0.1875
10  1  1  1  1  0  0  1  1  0  0  0  1  0  0  0  0    3.0       4  0.1875
11  1  1  1  1  0  0  1  1  1  0  0  0

...

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
3   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       1   0.125
4   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       4   0.125
5   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      15   0.125
6   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      28   0.125
7   0  0  0  0  0  1  0  0  0  1  0  1  1  1  1  1    1.0       2  0.1875
8   0  0  0  0  1  0  0  0  1  0  1  0  1  1  1  1    1.0       1  0.1875
9   0  0  0  0  0  1  0  0  1  1  0  0  1  1  1  1    1.0       1  0.1875
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       1  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      18  0.1875
2   0  0  0  0  0  1  0  0  1  1  1  0  0  1  1  1    2.0       1  0.1875
11  1  1  1  1  0  0  1  1  0  0  0  1  0  0  0  0    3.0       8  0.1875
12  1  1  1  0  0  1  1  1  1  0  0  0  0  0  0  0    3.0       1  0.1875
13  1  1  0  1  0  1  1  1  1  0  0  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
3   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       2   0.125
4   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       2   0.125
5   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      15   0.125
6   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      16   0.125
8   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0       1  0.1875
9   0  0  0  0  0  1  0  0  0  0  1  1  1  1  1  1    1.0       1  0.1875
10  0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0       1  0.1875
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0      10  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      23  0.1875
2   0  0  0  0  0  1  0  0  1  1  1  0  0  1  1  1    2.0       1  0.1875
7   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0       1    0.25
12  1  1  1  1  0  0  1  1  0  0  0  1  0  0  0  0    3.0      11  0.1875
13  1  1  1  0  0  1  1  1  1  0  0  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
3   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       3   0.125
4   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       3   0.125
5   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0      21   0.125
6   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      17   0.125
7   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       1  0.1875
8   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0       1  0.1875
10  0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0       1  0.1875
11  0  0  0  0  0  1  0  0  0  1  0  1  1  1  1  1    1.0       1  0.1875
12  0  0  0  0  0  1  0  0  0  0  1  1  1  1  1  1    1.0       1  0.1875
13  0  0  0  0  1  0  0  0  0  0  1  1  1  1  1  1    1.0       1  0.1875
19  0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       1    0.25
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       3  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
2   0  0  0  0  1  0  0  0  1  1  1  0  1  1  1  1    1.0       5   0.125
3   0  0  0  0  1  0  0  0  0  1  1  1  1  1  1  1    1.0       2   0.125
4   0  0  0  0  0  1  0  0  0  1  1  1  1  1  1  1    1.0       6   0.125
5   0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0      26   0.125
9   0  0  0  0  0  1  0  0  0  1  0  1  1  1  1  1    1.0       1  0.1875
10  0  0  0  0  0  1  0  0  0  0  1  1  1  1  1  1    1.0       2  0.1875
11  0  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1    1.0       1  0.1875
12  0  0  0  0  0  1  0  0  1  1  1  0  1  1  1  1    1.0       1    0.25
0   0  0  0  0  1  0  0  0  0  1  1  0  1  1  1  1    2.0       7  0.1875
1   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0      26  0.1875
7   0  0  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0       1    0.25
8   0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1    2.0       1  0.1875
13  1  1  1  1  0  0  1  1  0  0  0  1

({0: [],
  1: [],
  2: [],
  3: [[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
   [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]],
  4: [],
  5: [],
  6: [],
  7: [[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]],
  8: [],
  9: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: [],
  15: [],
  16: [],
  17: [],
  18: [],
  19: [],
  20: [],
  21: [],
  22: [],
  23: [],
  24: [],
  25: [],
  26: [],
  27: [],
  28: [],
  29: [],
  30: [],
  31: [],
  32: [],
  33: [],
  34: [],
  35: [],
  36: [],
  37: [],
  38: [],
  39: [],
  40: [],
  41: [],
  42: [],
  43: [],
  44: [],
  45: [],
  46: [],
  47: [],
  48: [[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]],
  49: [],
  50: [],
  51: [[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1]],
  52: [],
  53: [],
  54: [],
  55: [],
  56: [],
  57: [],
  58: [],
  59: [],
  60: [],
  61: [],
  62: [],
  63: [],
  64: [],
  65: [],
  66: [],
  67: [],
  68: [],
  69: [],
  70: [],
  71: [],
  72: [],
  73: [],
  74: [],
  75: [],
  76:

In [19]:
find_valid_y_3(df, num_reads=100, max_chain_strength=50, chain_break_fraction=True)

({0: [],
  1: [],
  2: [],
  3: [[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]],
  4: [],
  5: [],
  6: [],
  7: [],
  8: [[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
   [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]],
  9: [],
  10: [[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]],
  11: [],
  12: [],
  13: [],
  14: [],
  15: [],
  16: [],
  17: [],
  18: [],
  19: [],
  20: [],
  21: [],
  22: [],
  23: [[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]],
  24: [],
  25: [[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]],
  26: [],
  27: [],
  28: [],
  29: [],
  30: [],
  31: [],
  32: [],
  33: [[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]],
  34: [],
  35: [],
  36: [],
  37: [],
  38: [],
  39: [],
  40: [],
  41: [],
  42: [],
  43: [],
  44: [],
  45: [],
  46: [],
  47: [],
  48: [],
  49: [],
  50: [],
  51: [],
  52: [],
  53: [],
  54: [],
  55: [],
  56: [],
  57: [],
  58: [],
  59: [[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]],
  60: [],
  61: [[0, 

In [ ]:
c

In [25]:
def test_validity():
    df1 = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)
    new_y = np.array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 
test_validity()

[8 6 4 4]
[8 6 4 4]


In [72]:
def time_measurement(df):
    sum_time = 0
    annealing_time = 20
    for t1 in range(0, sum(df['LI'])+1):
        timeit_repeat = timeit.repeat("make_Hamiltonian(df, t1)", number=1, repeat=1, globals={"make_Hamiltonian": make_Hamiltonian, "df": df, "t1": t1})
        sum_time += timeit_repeat[0] + annealing_time
    return sum_time

In [73]:
#==========
#テストコード
#==========
def test_find_valid_y():
    df = pd.read_csv('../../input/ost6.csv', sep=',', index_col=0)
    true_t1 = sum(df['Y'] * df['LI'])
    valid_y_list, valid_y_num = find_valid_y(df,  num_reads=20)
    print(valid_y_list, valid_y_num)
    assert valid_y_num[true_t1] > 0
    
test_find_valid_y()

{0: [], 1: [], 2: [[1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0]], 3: [], 4: [], 5: [], 6: []} {0: 0, 1: 0, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0}


In [64]:
def test_validity():
    df1 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    new_y = np.array([1, 0, 0, 0, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 

#test_validity()

[2 2 0 1]
[2 2 0 1]
